In [34]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [35]:
IMG_SIZE = 180

In [36]:
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)


In [37]:
train_set = datagen_train.flow_from_directory("../dataset/ttte/train", target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 200 images belonging to 8 classes.


In [38]:
test_set = datagen_test.flow_from_directory("../dataset/ttte/validation", target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 40 images belonging to 8 classes.


In [39]:
train_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [40]:
test_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [41]:
STEP_SIZE_TRAIN = train_set.samples // train_set.batch_size
STEP_SIZE_TRAIN

6

In [42]:
STEP_SIZE_TEST = test_set.samples // test_set.batch_size
STEP_SIZE_TEST

1

In [43]:
data_augmentation = Sequential(
    [

    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
    ]
)


In [44]:
model = Sequential([
    layers.Input((IMG_SIZE, IMG_SIZE, 3)),
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(16,(3,3), padding='same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,(3,3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,(3,3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(len(train_set.class_indices))
])
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)       │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_2 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 180, 180, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 90, 90, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 90, 90, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 45, 45, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     3,965,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,989,672 (15.22 MB)

 Trainable params: 3,989,672 (15.22 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
opt = Adam(learning_rate=0.001)

callback = EarlyStopping(monitor='accuracy',
                         patience=20,
                         verbose=1,
                         mode="max",
                         baseline=0.4,
                          restore_best_weights=True
                         )


In [ ]:

model.compile(
            optimizer=opt,
              loss = tensorflow.keras.losses.CategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

history = model.fit(train_set,
                           epochs=50,
                           steps_per_epoch=STEP_SIZE_TRAIN,
                           validation_data=test_set, 
                          #  validation_steps=STEP_SIZE_TEST,
                           verbose=1,
                           callbacks=[callback]
                           )


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 343ms/step - accuracy: 0.0893 - loss: 2.0798 - val_accuracy: 0.1562 - val_loss: 2.0771
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.1562 - loss: 2.0803 - val_accuracy: 0.1562 - val_loss: 2.0774
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 247ms/step - accuracy: 0.1250 - loss: 2.0799 - val_accuracy: 0.0938 - val_loss: 2.0825
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.1250 - loss: 2.0788 - val_accuracy: 0.0938 - val_loss: 2.0825
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 258ms/step - accuracy: 0.1250 - loss: 2.0798 - val_accuracy: 0.1562 - val_loss: 2.0783
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1250 - loss: 2.0776 - val_accuracy: 0.0625 - val_loss: 2.0814
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 250ms/step - accuracy: 0.1369 - loss: 2.0804 - val_accuracy: 0.1562 - val_loss: 2.0798
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.0625 - loss: 2.0758 - val_accuracy: 0.1250 - val_loss: 2.0